<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/ResNet101%20without%20hidden%20dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 34.7 MB/s 
     |████████████████████████████████| 166 kB 70.7 MB/s 
     |████████████████████████████████| 182 kB 67.6 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 166 kB 74.5 MB/s 
     |████████████████████████████████| 162 kB 64.3 MB/s 
     |████████████████████████████████| 162 kB 61.8 MB/s 
     |████████████████████████████████| 158 kB 68.3 MB/s 
     |████████████████████████████████| 157 kB 74.5 MB/s 
     |████████████████████████████████| 157 kB 73.5 MB/s 
     |████████████████████████████████| 157 kB 75.5 MB/s 
     |████████████████████████████████| 157 kB 72.5 MB/s 
     |████████████████████████████████| 157 kB 61.0 MB/s 
     |████████████████████████████████| 157 kB 74.8 MB/s 
     |████████████████████████████████| 157 kB 77.2 MB/s 
     |████████████████████████████████| 156 kB 69.8 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [5]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [8]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [15]:
resnet101 = tf.keras.applications.ResNet101(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg')
model_resnet101=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  resnet101,
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
resnet101.trainable=False
model_resnet101.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 resnet101 (Functional)      (None, 2048)              42658176  
                                                                 
 dropout_2 (Dropout)         (None, 2048)              0         
                                                                 
 dense_3 (Dense)             (None, 5)                 10245     
                                                                 
Total params: 42,668,421
Trainable params: 10,245
Non-trainable params: 42,658,176
_________________________________________________________________


In [16]:
model_resnet101.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [17]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "resnet101_without hidden dense layer",
)

In [18]:
%%time
epochs=40
history = model_resnet101.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.4205 - accuracy: 0.4318

wandb: Adding directory to artifact (/content/wandb/run-20221105_064244-5w90gczg/files/model-best)... Done. 1.1s


22/22 [==============================] - 58s 2s/step - loss: 1.4205 - accuracy: 0.4318 - val_loss: 1.3311 - val_accuracy: 0.3886
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 1.3303 - accuracy: 0.4901

wandb: Adding directory to artifact (/content/wandb/run-20221105_064244-5w90gczg/files/model-best)... Done. 1.2s


22/22 [==============================] - 52s 2s/step - loss: 1.3303 - accuracy: 0.4901 - val_loss: 1.2805 - val_accuracy: 0.5200
Epoch 3/40
22/22 [==============================] - ETA: 0s - loss: 1.2619 - accuracy: 0.5284

...

22/22 [==============================] - 49s 2s/step - loss: 0.9854 - accuracy: 0.6307 - val_loss: 0.9826 - val_accuracy: 0.6400
Epoch 37/40
22/22 [==============================] - 6s 261ms/step - loss: 0.9566 - accuracy: 0.6364 - val_loss: 0.9845 - val_accuracy: 0.6514
Epoch 38/40
22/22 [==============================] - ETA: 0s - loss: 0.9706 - accuracy: 0.6392

wandb: Adding directory to artifact (/content/wandb/run-20221105_064244-5w90gczg/files/model-best)... Done. 1.3s


22/22 [==============================] - 49s 2s/step - loss: 0.9706 - accuracy: 0.6392 - val_loss: 0.9717 - val_accuracy: 0.6629
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 0.9529 - accuracy: 0.6506

wandb: Adding directory to artifact (/content/wandb/run-20221105_064244-5w90gczg/files/model-best)... Done. 1.2s


22/22 [==============================] - 49s 2s/step - loss: 0.9529 - accuracy: 0.6506 - val_loss: 0.9690 - val_accuracy: 0.6571
Epoch 40/40
22/22 [==============================] - 6s 264ms/step - loss: 0.9579 - accuracy: 0.6520 - val_loss: 0.9698 - val_accuracy: 0.6457
CPU times: user 18min 49s, sys: 1min 3s, total: 19min 53s
Wall time: 27min 37s


In [13]:
wandb.finish()

accuracy,▁▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇████▇▇██
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▆▅▅▅▅▄▄▄▄▄▄▃▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁
val_accuracy,▁▁▂▂▁▂▂▂▂▁▁▁▁▁▃▂▂▂▂▃▂▄▄▄▂▅▅▆▄▄▄▆▅▅▇▇▇▆▆█
val_loss,█▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▃▂▂▂▂▂▁▁▁▁▁▁
accuracy,0.66619
best_epoch,39
best_val_loss,0.90209
epoch,39
loss,0.90253
val_accuracy,0.73714


In [14]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
